# Notebook 1 : Web Scraping

L'objectif de ce notebook est de scraper le site https://www.cocktailmag.fr/ pour y récupérer l'ensemble des cocktails

## Étape 1 : Déterminer les informations que l'on recherche

Sur le site, on retrouve la page [/recette-cocktail](https://www.cocktailmag.fr/recette-cocktail) qui contient une liste de cocktail avec déjà quelques informations :

<img src="../img/web_scraping/site.png" alt="Site" width="600"/>

Sur chacun de ces cocktails, on peut cliquer dessus pour accéder au détail du cocktail :

<img src="../img/web_scraping/cocktail.png" alt="Cocktail" width="600"/>

Ainsi avec ces 2 sources de données, on va pouvoir récupérer les informations suivantes :

- Nom du cocktail
- Image du cocktail (lien)
- Description courte
- Caractéristiques
- Ingrédients
- Recette
- A propos

Nous allons donc construire un dataframe Pandas avec ces informations, avec une ligne par cocktail. Nous chargerons ensuite ce dataframe dans un fichier CSV pour pouvoir l'utiliser dans les prochains notebooks.

## Étape 2 : Scraper le site web

La page avec une liste de cocktails contient 15 cocktails par page, et on peut naviguer entre les pages. Un point qui nous arrange est que la navigation peut se faire simplement avec l'argument `start` dans l'URL. Par exemple, pour accéder à la page 2, il suffit de rajouter `?start=15` à l'URL.

Nous allons donc nous servir de cette information pour scraper l'ensemble des cocktails.

Pour récupérer toutes les informations de chaque cocktail, notre algorithme va procéder de la manière suivante :

1. Récupérer la liste des cocktails sur la page
2. Pour chaque cocktail, récupérer les informations
3. Récupérer le lien vers le détail du cocktail
4. Récupérer les informations du détail du cocktail

Et nous recommencerons cette opération pour chaque page jusqu'à ce qu'il n'y ait plus de cocktails.

Pour cela, nous allons utiliser la librairie `requests` pour récupérer le contenu des pages, et `BeautifulSoup` pour parser le contenu et récupérer les informations qui nous intéressent.

In [ ]:
import requests
from bs4 import BeautifulSoup

names = []
img_links = []
descriptions = []
tags = []
ingredients = []
recipes = []
abouts = []

def get_cocktails(start):
    URL = "https://www.cocktailmag.fr/recette-cocktail?start=" + str(start)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    cocktails = soup.find('div', attrs={'class' :"com-content-category-blog__items blog-items boxed columns-3"})

    cocktail_list = cocktails.find_all('div', attrs={'class' :"com-content-category-blog__item blog-item"})
    
    for cocktail in cocktail_list:
        # ===============
        #  Première page
        # ===============

        # Nom du cocktail
        name = cocktail.find('h2').text.strip()

        # Lien vers l'image du cocktail
        img_link = cocktail.find('img')['src']

        # Description du cocktail
        description = cocktail.find('div', attrs={'class' :"itemintro"}).text.strip()

        # Tags du cocktail
        tag = cocktail.find('ul', attrs={'class' :"tags list-inline"})
        tag = "; ".join([t.text.strip() for t in tag.find_all('li')])


        # =======================
        #  Page détails cocktail
        # =======================
        URL_cocktail = cocktail.find('a')['href']
        page_cocktail = requests.get("https://www.cocktailmag.fr/" + URL_cocktail)
        soup_cocktail = BeautifulSoup(page_cocktail.content, "html.parser")

        # Ingrédients du cocktail
        ingredient = "; ".join([i.text for i in soup_cocktail.find_all('div', attrs={'class' :"ingredients"})])

        # Recette du cocktail
        recipe_div = soup_cocktail.find('div', attrs={'class' :"itembottom"})
        recipe = recipe_div.find('ol')
        recipe = "; ".join([step.text.strip() for step in recipe.find_all('li')])

        # A propos / Histoire du cocktail
        # Cas spécifique pour le cocktail "Spritz" qui n'a pas de div "itemhistory"
        if name == "Spritz":
            about = "; ".join([
                    "L’histoire du Spritz remonte au XIXe siècle, lorsque l’Empire austro-hongrois occupait le nord de l’Italie, notamment la Vénétie et la Lombardie. Les soldats et fonctionnaires autrichiens, peu habitués aux vins italiens plus forts que ceux de leur pays, demandaient aux aubergistes d’adoucir ces vins en les allongeant avec un peu d’eau gazeuse.",
                    "Le mot \"Spritz\" vient d’ailleurs du verbe allemand \"spritzen\", qui signifie \"asperger\" ou \"arroser\", en référence à cette habitude d’ajouter de l’eau pétillante au vin."
            ])
        # Cas spécifique pour les cocktail qui n'ont aucune section "A propos"
        elif name in ["Blackjack", "Bedroom Farce", "Marama Rum Punch"]:
            about = ""
        # Cas général
        else:
            about_divs = soup_cocktail.find_all('div', attrs={'class' :"itemhistory"})
            target_div = None
            for div in about_divs:
                if div.find('p'):
                    target_div = div
            about = "; ".join([p.text.strip() for p in target_div.find_all('p')])


        # Insertion dans les listes
        names.append(name)
        img_links.append(img_link)
        descriptions.append(description)
        tags.append(tag)
        ingredients.append(ingredient)
        recipes.append(recipe)
        abouts.append(about)

    return len(cocktail_list)


In [152]:
start = 0
while get_cocktails(start) == 15:
    start += 15 

## Étape 3 : Création du dataframe

Désormais nous pouvons enregistrer les données dans un dataframe Pandas.

In [153]:
import pandas as pd

data = {
    "name": names,
    "img_link": img_links,
    "description": descriptions,
    "tags": tags,
    "ingredients": ingredients,
    "recipe": recipes,
    "about": abouts
}

df = pd.DataFrame.from_dict(data)

# Étape 4 : Vérification des données

Avant d'enregistrer les données dans un fichier CSV, nous allons vérifier que les données récupérées sont correctes.

Nous allons notamment vérifier s'il n'y a aucun doublon et s'il n'y a pas de données manquantes.

In [154]:
print(df['name'].duplicated().sum())

0


In [155]:
print(df.isnull().sum())

name           0
img_link       0
description    0
tags           0
ingredients    0
recipe         0
about          0
dtype: int64


# Étape 5 : Enregistrement des données dans un fichier CSV

Enfin nous pouvons enregistrer les données dans un fichier CSV pour pouvoir les utiliser dans les prochains notebooks.

In [ ]:
df.to_csv("../dataset/cocktails.csv", index=False)